Continues from A006h

In [1]:
import os
import sys
import warnings
import random
import copy
import pickle
import glob

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append('../common')
import data_io_utils
import paths
import constants
import utils

sys.path.append('../A003_policy_optimization/')
import models
import A003_common

import A006_common
from unirep import babbler1900 as babbler
import sequence_selection


%reload_ext autoreload
%autoreload 2

## Sync data.

We only need to sync the selected sequence folders.

In [2]:
!aws s3 cp s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/ ../../data/s3/chip_1/simulated_annealing/GFP/ --exclude "*" --include "*selected_seqs*" --recursive

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GF

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-3ea5f6e-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-3ea5f6e-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-3ea5f6e-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-3ea5f6e-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-SmallTrust-2e8818d2-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-SmallTrust-2e8818d2-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-SmallTrust-2e8818d2-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-SmallTrust-2e8818d2-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-01-SmallTrust-3f4221b8-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-01-SmallTrust-3f4221b8-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-01-SmallTrust-3f4221b8-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-01-SmallTrust-3f4221b8-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simu

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-04-30cb4ce7-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-04-30cb4ce7-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-009

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-02-f795278-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-02-f795278-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-02-f795278-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-02-f795278-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-03-2a95c17b-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0024-03-2a95c17b-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_2-0

...

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-01-15947f78-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-01-15947f78-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-02-1d0c4863-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-02-1d0c4863-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-02-1d0c4863-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0024-02-1d0c4863-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Rando

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-01-bb13a0-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-01-bb13a0-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-01-bb13a0-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-01-bb13a0-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-02-311cbde3-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-02-311cbde3-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-03-108b3ec7-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-03-108b3ec7-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0024-02-bb99846-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0024-02-bb99846-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0024-01-1723ee07-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0024-01-1723ee07-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-00-SmallTrust-272dab56-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-00-SmallTrust-272dab56-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-00-2277bff5-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Random_Init_1-0096-00-2277bff5-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-00-23fe40b5-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-00-23fe40b5-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-00-23fe40b5-se

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-2262da3b-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-2262da3b-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-2262da3b-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-2262da3b-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-SmallTrust-31616a94-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-02-SmallTrust-31616a94-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/GFP/GFP_SimAnneal-OneHot-0096-03-e03b927-selected_seqs/A006j_w

In [3]:
!aws s3 cp s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/ ../../data/s3/chip_1/simulated_annealing/beta_lactamase/ --exclude "*" --include "*selected_seqs*" --recursive

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-01-3a0e3d4-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-01-3a0e3d4-selected_seqs/pairwise_levenshtein

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-04-4502d3-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-04-4502d3-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-03-3764e943-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-03-3764e943-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-03-3764e943-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0024-03-3764e943-selected_seqs/top_seq_and_traj_plot.png
download: s

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-04-30cb4ce7-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-04-30cb4ce7-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Global_Init_1-0096-03-12da09f-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-03-2a95c17b-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-03-2a95c17b-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-04-36079a2c-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-04-36079a2c-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-03-2a95c17b-selected_seqs/sr_vs_nsr_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0024-03-2a95c17b-selected_seqs/sr_vs_nsr_plot.png
download: s3://low-n-protein-engineering-public/c

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-ET_Random_Init_1-0096-03-33bfb65e-selected_seqs/pairwise_levenshtein_distance_plot.png
download:

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-LargeMut-OneHot-0096-00-75e84ab-selected_seqs/seq_similarity_summary_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-LargeMut-OneHot-0096-00-75e84ab-selected_seqs/seq_similarity_summary_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-LargeMut-OneHot-0096-00-75e84ab-selected_seqs/selected_seqs_df.pkl to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-LargeMut-OneHot-0096-00-75e84ab-selected_seqs/selected_seqs_df.pkl
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-00-15ce00a7-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-00-15ce00a7-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-02-1d0c4863-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-02-1d0c4863-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-04-1989c098-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-04-1989c098-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-04-1989c098-selected_seqs/pairwise_levenshtein_distance_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0024-04-1989c098-selected_seqs/pairwise_levenshtein_distance_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_

download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-04-1a4d5748-selected_seqs/A006j_wt_pred_qfunc.npy to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-04-1a4d5748-selected_seqs/A006j_wt_pred_qfunc.npy
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-03-108b3ec7-selected_seqs/qfunc_hist_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-03-108b3ec7-selected_seqs/qfunc_hist_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-03-108b3ec7-selected_seqs/top_seq_and_traj_plot.png to ../../data/s3/chip_1/simulated_annealing/beta_lactamase/BLAC_SimAnneal-OneHot-0096-03-108b3ec7-selected_seqs/top_seq_and_traj_plot.png
download: s3://low-n-protein-engineering-public/chip_1/simulated_annealing/beta_lactamase/BLAC_

In [4]:
GFP_selected_seqs_dir = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 'simulated_annealing', 'GFP')
BLAC_selected_seqs_dir = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 'simulated_annealing', 'beta_lactamase')

## Load the sequence allocation table. 

This is copied and pasted from here ("Tabular Seq Allocation" tab): https://docs.google.com/spreadsheets/d/1UVfD0TYKvWi2CPM9tec1qtry053UarqxRvx8BKR7Ytw/edit#gid=533377689

In [5]:
seq_alloc_df = pd.read_csv('Chip_1_Sequence_Allocation.txt', sep='\t')
seq_alloc_df.head()

,protein,base_model,n_train,trust_radius,seed,seqs_allocated
0,GFP,ET_Global_Init_1,24,15,0,300
1,GFP,ET_Global_Init_1,24,15,1,300
2,GFP,ET_Global_Init_1,24,15,2,300
3,GFP,ET_Global_Init_1,24,15,3,300
4,GFP,ET_Global_Init_1,24,15,4,300


In [6]:
selected_seq_dfs = []
for i,r in seq_alloc_df.iterrows():
    if r['n_train'] == 'LARGE_N ??':
        continue # Never did these experiments.
        
    if r['n_train'] == 'Grigory\'s Special Set':
        gk_df = pd.read_csv('GK_Special_Set_20190812_df_version2_gfp_greedy.csv')
        gk_df['protein'] = 'GFP'
        selected_seq_dfs.append(gk_df)
        
        continue # for now
    
    protein = r['protein']
    glob_exp = '*' + r['base_model'] + '-00' + str(r['n_train']) + '-0' + r['seed'] + '-*-selected_seqs'
    
    # Grab the directory containing the sequences selected from simulated annealing (see A006h)
    if protein == 'GFP':
        selected_seqs_dir = glob.glob(os.path.join(GFP_selected_seqs_dir, glob_exp))
                
        if int(r['trust_radius']) == 15:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('SmallTrust' not in s)]
        elif int(r['trust_radius']) == 7:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('SmallTrust' in s)]
        else:
            assert False, 'unexpected trust radius'
                        
        assert len(selected_seqs_dir) == 1
        
        selected_seqs_dir = selected_seqs_dir[0]
        
    elif protein == 'BLAC':
        
        selected_seqs_dir = glob.glob(os.path.join(BLAC_selected_seqs_dir, glob_exp))
                
        if int(r['trust_radius']) == 15:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('LargeMut' in s)]
        elif int(r['trust_radius']) == 7:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('LargeMut' not in s)]
        else:
            assert False, 'unexpected trust radius'
            
        assert len(selected_seqs_dir) == 1
        
        selected_seqs_dir = selected_seqs_dir[0]
        
    print(selected_seqs_dir)
    seq_file = os.path.join(selected_seqs_dir, 'selected_seqs_df.pkl')
    seq_df = pd.read_pickle(seq_file)
    seq_df['protein'] = r['protein']
    
    selected_seq_dfs.append(seq_df.iloc[:r['seqs_allocated']])    

aggregated_selected_seq_df = pd.concat(selected_seq_dfs, axis=0, sort=False)        

/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-00-3e721641-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-01-3a0e3d4-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-02-31e54146-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-03-3764e943-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0024-04-4502d3-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-00-3ea5f6e-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing/GFP/GFP_SimAnneal-ET_Global_Init_1-0096-01-2db7371-selected_seqs
/notebooks/analysis/common/../../data/s3/chip_1/simulated_annealing

In [7]:
print(aggregated_selected_seq_df.shape)
aggregated_selected_seq_df.head()

(24137, 8)


,id,seq_idx,trajectory_idx,predicted_fitness,ensemble_predicted_fitness,seq,protein,pred_score
0,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,615.0,1516.0,0.835807,"[0.8386957, 0.8833945, 0.6869724, 0.7490394, 0...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKMTLKF...,GFP,NaN
1,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,250.0,922.0,0.831929,"[0.83363, 0.8859961, 0.6875845, 0.74543333, 0....",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,GFP,NaN
2,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,250.0,195.0,0.830867,"[0.8327829, 0.87653375, 0.6890327, 0.73599446,...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,GFP,NaN
3,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,354.0,1847.0,0.828640,"[0.82065207, 0.8765222, 0.6913246, 0.747247, 0...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATMGKLTLKF...,GFP,NaN
4,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,933.0,163.0,0.825671,"[0.83166367, 0.8713553, 0.69725585, 0.7542225,...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDASYGKMTIKF...,GFP,NaN


In [8]:
aggregated_selected_seq_df.tail()

,id,seq_idx,trajectory_idx,predicted_fitness,ensemble_predicted_fitness,seq,protein,pred_score
657,42{fcorrect_boundaries_indel_42randomseed_6mut...,NaN,NaN,NaN,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKMTIKF...,GFP,1.200250
658,43{fcorrect_boundaries_indel_42randomseed_6mut...,NaN,NaN,NaN,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDASYGHLNLKF...,GFP,1.235714
659,44{fcorrect_boundaries_indel_42randomseed_6mut...,NaN,NaN,NaN,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLALKF...,GFP,1.301181
660,single_adversarial_10greedy_run,NaN,NaN,NaN,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDSTYGKLTLKE...,GFP,3.832737
661,single_most_freq_2greedy_run,NaN,NaN,NaN,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGALTLKF...,GFP,1.201304


## Export

In [9]:
output_file = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 
        'A006i_aggregated_selected_seqs_for_cloning_validation.pkl')
aggregated_selected_seq_df.to_pickle(output_file)

# Post publication: Commenting out original sync code as it will not work to the read-only bucket.
#data_io_utils.sync_local_path_to_s3(output_file, is_single_file=True)